In [1]:
import os, re
import pandas as pd
from utils import get_files_absolute_path_from_dir, connect_postgres, sql_to_dataframe

In [2]:
path = os.path.join(os.getcwd(), 'all_queries', 'updated_queries')
files = os.listdir(path)
print(path)
queries = get_files_absolute_path_from_dir(path)

/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries
Total files: 99
First few files...
['/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-08.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-79.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-06.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-81.sql', '/home/alfio/Desktop/DataWarehouse/TPC_DS/tpcds-benchmark/all_queries/updated_queries/query-30.sql']


In [3]:
pattern = r"\bwhere\b\s+.*?(?=\b(?:group\s+by|order\s+by|limit|$)\b)"

sql_query = """
SELECT id, name FROM users
WHERE age > 25 AND country = 'USA' ORDER BY name;
"""

# Case-insensitive match
matches = re.findall(pattern, sql_query, re.IGNORECASE)

# Output the matches
for match in matches:
    print(match)

WHERE age > 25 AND country = 'USA' 


In [4]:
split_pattern = r"\s*\b(AND|OR|WHERE)\b\s*"
conditions = re.split(split_pattern, ciao, flags=re.IGNORECASE)
conditions = [cond.strip() for cond in conditions if cond.strip().upper() not in ['WHERE', 'AND', 'OR', '']]
print(conditions)

NameError: name 'ciao' is not defined

In [5]:
pattern = r"\bWHERE\b\s+.*?(?=\b(?:GROUP\s+BY|ORDER\s+BY|LIMIT|INTERSECT|UNION|SELECT|$)\b)"
split_pattern = r"\s*\b(AND|OR|WHERE)\b\s*"

total_conditions = 0
all_conditions = []
total_wheres = 0
total_matches = 0
for query in queries:
    with open(query, "r") as f:
        all_lines = []
        for line in f:
            cleaned_line = line.strip()
            if cleaned_line != '':
                all_lines.append(cleaned_line)
                
        one_line_query = ' '.join(all_lines)
        matches = re.findall(pattern, one_line_query, re.IGNORECASE)
        wheres = re.findall(r"\bWHERE\b", one_line_query, re.IGNORECASE)
        print(len(wheres) != len(matches))
        total_wheres += len(wheres)
        total_matches += len(matches)
        print(f"Query {query[-6:-4]} returned {len(matches)} matches:")
        for match in matches:
            conditions = re.split(split_pattern, match, flags=re.IGNORECASE)
            conditions = [cond.strip() for cond in conditions if cond.strip().upper() not in ['WHERE', 'AND', 'OR', '']]
            conditions = [re.sub("\s{2,}", " ", cond) for cond in conditions]
            all_conditions.extend(conditions)
            print("\n".join(conditions))
            print(len(conditions))
            total_conditions += len(conditions)
            print()
            
            
print(f"Total no of conditions {total_conditions}")
print(f"Total no of wheres {total_wheres}")
print(f"Total no of matches {total_matches}")
        #matches = re.findall(pattern, content, re.IGNORECASE)
        #print(matches)

False
Query 08 returned 3 matches:
substr(ca_zip,1,5) IN ( '19100','41548','51640','49699','88329','55986', '85119','19510','61020','95452','26235', '51102','16733','42819','27823','90192', '31905','28865','62197','23750','81398', '95288','45114','82060','12313','25218', '64386','46400','77230','69271','43672', '36521','34217','13017','27936','42766', '59233','26060','27477','39981','93402', '74270','13932','51731','71642','17710', '85156','21679','70840','67191','39214', '35273','27293','17128','15458','31615', '60706','67657','54092','32775','14683', '32206','62543','43053','11297','58216', '49410','14710','24501','79057','77038', '91286','32334','46298','18326','67213', '65382','40315','56115','80162','55956', '81583','73588','32513','62880','12201', '11592','17014','83832','61796','57872', '78829','69912','48524','22016','26905', '48511','92168','63051','25748','89786', '98827','86404','53029','37524','14039', '50078','34487','70142','18697','40129', '60642','42810','62667','57183'

In [6]:
all_conditions

["substr(ca_zip,1,5) IN ( '19100','41548','51640','49699','88329','55986', '85119','19510','61020','95452','26235', '51102','16733','42819','27823','90192', '31905','28865','62197','23750','81398', '95288','45114','82060','12313','25218', '64386','46400','77230','69271','43672', '36521','34217','13017','27936','42766', '59233','26060','27477','39981','93402', '74270','13932','51731','71642','17710', '85156','21679','70840','67191','39214', '35273','27293','17128','15458','31615', '60706','67657','54092','32775','14683', '32206','62543','43053','11297','58216', '49410','14710','24501','79057','77038', '91286','32334','46298','18326','67213', '65382','40315','56115','80162','55956', '81583','73588','32513','62880','12201', '11592','17014','83832','61796','57872', '78829','69912','48524','22016','26905', '48511','92168','63051','25748','89786', '98827','86404','53029','37524','14039', '50078','34487','70142','18697','40129', '60642','42810','62667','57183','46414', '58463','71211','46364'

In [7]:
cur = connect_postgres("tpcds")

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'tpcds', 'host': 'localhost', 'port': '25433', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 

You are connected to -  ('PostgreSQL 17.0 (Debian 17.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) 



In [10]:
df = sql_to_dataframe(cur, "SELECT attname from PG_STATS WHERE schemaname = 'public';", ["column"])

In [16]:
no_operations = {}
for column in list(df['column']):
    no_operations[column] = 0
    
for column in list(df['column']):
    for condition in all_conditions:
        if column in condition:
            no_operations[column]+= 1
            
no_operations

{'ca_address_sk': 41,
 'ca_address_id': 0,
 'ca_street_number': 0,
 't_second': 0,
 't_am_pm': 0,
 'ca_street_name': 0,
 'ca_street_type': 0,
 'ca_suite_number': 0,
 'ca_city': 3,
 'ca_county': 7,
 'ca_state': 12,
 't_shift': 0,
 't_sub_shift': 0,
 'wp_autogen_flag': 0,
 'ca_zip': 7,
 'ca_country': 5,
 'ca_gmt_offset': 12,
 'ca_location_type': 0,
 'cd_demo_sk': 15,
 'cd_gender': 4,
 'cd_marital_status': 13,
 'cd_education_status': 12,
 'cd_purchase_estimate': 0,
 'cd_credit_rating': 0,
 'cd_dep_count': 0,
 'cd_dep_employed_count': 0,
 'cd_dep_college_count': 0,
 't_time_sk': 13,
 't_time_id': 0,
 't_time': 15,
 't_hour': 10,
 't_minute': 8,
 't_meal_time': 2,
 'i_item_sk': 80,
 'i_item_id': 7,
 'i_rec_start_date': 0,
 'i_rec_end_date': 0,
 'i_item_desc': 0,
 'i_current_price': 10,
 'i_wholesale_cost': 0,
 'i_brand_id': 4,
 'wp_customer_sk': 0,
 'i_brand': 12,
 'i_class_id': 4,
 'i_class': 11,
 'i_category_id': 4,
 'i_category': 38,
 'i_manufact_id': 10,
 'c_customer_id': 0,
 'c_current